In [163]:
import pandas as pd
import numpy as np

In [164]:
sample_items_meli = pd.read_csv('sample_items_meli.csv')
# Caso queira limitar o número de amostras
# sample_items_meli = sample_items_meli.iloc[:1500]

In [165]:
categorias_distintas = len(set(sample_items_meli['ITE_ITEM_DOM_DOMAIN_ID']))
categorias_distintas

832

In [166]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import os

# Função para obter produtos similares do mesmo cluster
def obter_produtos_similares(df, item_id, top_n=5):
    # Obtém o cluster do item especificado
    cluster_id = df.loc[df['item_id'] == item_id, 'cluster'].values[0]
    # Seleciona todos os itens que estão no mesmo cluster
    itens_similares = df[df['cluster'] == cluster_id]
    # Exclui o próprio item da lista de similares
    itens_similares = itens_similares[itens_similares['item_id'] != item_id]
    # Retorna os top_n itens similares
    return itens_similares.head(top_n)


In [167]:
# Função para carregar e achatar as imagens
def carregar_e_achatar_imagens(df, modo_cor='rgb', tamanho_imagem=(64, 64)):
    imagens = []
    ids = []
    for idx, row in df.iterrows():
        caminho_img = row['thumbnail_path']
        try:
            # Abre a imagem do caminho especificado
            img = Image.open(caminho_img)
            # Redimensiona a imagem para um tamanho padrão para uniformizar
            img = img.resize(tamanho_imagem)
            if modo_cor == 'grayscale':
                # Converte a imagem para escala de cinza
                img = img.convert('L')
                # Converte a imagem em um array numpy e achata em um vetor 1D
                img_array = np.array(img).flatten()
            else:
                # Garante que a imagem está em modo RGB
                img = img.convert('RGB')
                # Converte a imagem em um array numpy e achata em um vetor 1D
                img_array = np.array(img).flatten()
            # Adiciona o vetor de imagem à lista de imagens
            imagens.append(img_array)
            # Armazena o item_id correspondente
            ids.append(row['item_id'])
        except Exception as e:
            # Em caso de erro ao carregar a imagem, imprime o erro e continua
            print(f"Erro ao carregar a imagem {caminho_img}: {e}")
            continue
    # Retorna um array numpy das imagens e a lista de ids
    return np.array(imagens), ids


# === Recomendação usando imagens RGB ===
# Carrega e achata as imagens em RGB
imagens_rgb, ids_rgb = carregar_e_achatar_imagens(sample_items_meli, modo_cor='rgb', tamanho_imagem=(128, 128))
print(len(imagens_rgb))
print(len(ids_rgb))
print(imagens_rgb[0])

15270
15270
[255 255 255 ... 255 255 255]


In [168]:
# Função para realizar PCA e clustering nas imagens
def realizar_pca_clusterizacao(dados_imagem, n_componentes=50, n_clusters=10):
    # Normaliza os dados para ter média zero e variância unitária
    scaler = StandardScaler()
    dados_imagem_escalados = scaler.fit_transform(dados_imagem)
    
    # Aplica PCA para reduzir a dimensionalidade dos dados
    pca = PCA(n_components=n_componentes)
    dados_imagem_pca = pca.fit_transform(dados_imagem_escalados)
    
    # Aplica KMeans para agrupar os dados em clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(dados_imagem_pca)
    
    # Retorna os clusters e os dados transformados pelo PCA
    return clusters, dados_imagem_pca


# Realiza PCA e clustering nas imagens RGB
clusters_rgb, pca_rgb = realizar_pca_clusterizacao(
    imagens_rgb,
    n_componentes=50,
    n_clusters=categorias_distintas//2)
print(len(clusters_rgb))
print(len(pca_rgb))
print(clusters_rgb.shape)
print(pca_rgb.shape)
print(pca_rgb[0])
print(clusters_rgb)

15270
15270
(15270,)
(15270, 50)
[-88.33060979  10.06263592 -50.72166335  -0.1567997   23.40141498
  58.67339205  24.27689025  30.82815064  -5.64904308   3.40754491
 -16.70789606  20.39696917   5.16768817  29.81579808 -40.79715543
  15.48545147  41.53240502  -0.24228733  27.32784216  -1.66513257
   4.11372586   2.16101615  11.09234182 -40.60293481  -4.12806753
 -20.9475233  -19.09272873  19.60495209   3.78364263  -9.77476853
  -8.52166056 -24.48688552 -17.989191     3.37373534   8.26858855
 -13.94263277  14.26715944   7.79568105   7.93224112   9.79092734
  -9.19855238  -0.99547207  12.32022359  15.08363646  -1.2495268
  -6.81078272   1.83001223  13.77876264 -18.78255832   9.98784701]
[107 300 247 ... 367 272  26]


In [169]:
# Adiciona os clusters ao DataFrame original
sample_items_meli_rgb = sample_items_meli.copy()
sample_items_meli_rgb['cluster'] = pd.Series(clusters_rgb, index=sample_items_meli_rgb.index)
sample_items_meli_rgb

,item_id,ITE_ITEM_TITLE,ITE_ITEM_DOM_DOMAIN_ID,buyer_ID,thumbnail_path,cluster
0,MLB4113179452,Balança Digital Portátil Até 50kg Mão Trava Ma...,MLB-PORTABLE_HANGING_SCALES,6,item_thumbs/926335-MLB72143164750_102023.jpeg,107
1,MLB3563191005,Câmbio Traseiro Shimano Altus Rd-m310 7v E 8v ...,MLB-BICYCLE_REAR_DERAILLEURS,49,item_thumbs/647822-MLU74334909051_012024.jpeg,300
2,MLB4485034748,Líquido Selante Slime Pneus Com Câmara 237ml B...,MLB-TIRE_SEALANTS,384,item_thumbs/672634-MLA79695224671_102024.jpeg,247
3,MLB1902910378,Singestar Comprimidos Anticoncepcional Cães E ...,MLB-ANIMAL_AND_PET_PRODUCTS,458,item_thumbs/841051-MLB76169731299_052024.jpeg,313
4,MLB3613377213,Par Pneu Mtb Michelin Force Aro 29 29x2.25,MLB-BICYCLE_TIRES,490,item_thumbs/825267-MLU75289032373_032024.jpeg,161
...,...,...,...,...,...,...
15265,MLB4578436388,1 Orlists - Fórmula Concentrada 120mg - C/ 90 ...,MLB-SUPPLEMENTS,453200,item_thumbs/907840-MLB75421682872_042024.jpeg,254
15266,MLB3832505417,Tadalaeuro 20mg Aumente Seu Desempenho Deforma...,MLB-SUPPLEMENTS,133353,item_thumbs/722247-MLB78769948837_082024.jpeg,179
15267,MLB3608376035,Nivea Creme Para As Mãos Reparação 75 G,MLB-BODY_SKIN_CARE_PRODUCTS,455603,item_thumbs/902156-MLU75568600365_042024.jpeg,367
15268,MLB2889297520,Máquina De Barbear Recarregável Kemei Double B...,MLB-HAIR_CLIPPERS_ELECTRIC_SHAVERS_AND_HAIR_TR...,717643,item_thumbs/639058-MLU77353751725_062024.jpeg,272


In [170]:
# Exemplo: Obtém produtos similares para um item específico usando clusters RGB
item_id_exemplo = sample_items_meli_rgb['item_id'].iloc[42]
produtos_similares_rgb = obter_produtos_similares(sample_items_meli_rgb, item_id_exemplo)
print(f"Produtos similares (RGB) para o item {item_id_exemplo}:\n", produtos_similares_rgb)

Produtos similares (RGB) para o item MLB2897190943:
             item_id                                     ITE_ITEM_TITLE  \
1171   MLB808538747  4 Buchas Da Bandeja Inferior Suspensão Diantei...   
1209  MLB2684324151  Varetas De Fibra De Algodão Preta 25cm 3mm Aro...   
1285  MLB4716454924  Para Sl7 Stem Garmin Bryton Mount Holder, Bici...   
1360  MLB2918318707  Banco Selim Absolute Gel Elastômero Super Conf...   
1508  MLB3376148905  Peneira Multiprocessador 3 Engrenagens  - Phil...   

                                ITE_ITEM_DOM_DOMAIN_ID  buyer_ID  \
1171       MLB-VEHICLE_SUSPENSION_CONTROL_ARM_BUSHINGS    109080   
1209                                MLB-REED_DIFFUSERS    112360   
1285                             MLB-CYCLING_COMPUTERS    120920   
1360                                 MLB-BICYCLE_SEATS    127466   
1508  MLB-HOME_APPLIANCES_ACCESSORIES_AND_REPLACEMENTS    141021   

                                     thumbnail_path  cluster  
1171  item_thumbs/691461-MLB31

In [171]:
# === Recomendação usando imagens em escala de cinza ===
# Carrega e achata as imagens em escala de cinza
imagens_gray, ids_gray = carregar_e_achatar_imagens(sample_items_meli, modo_cor='grayscale', tamanho_imagem=(128, 128))

# Realiza PCA e clustering nas imagens em escala de cinza
clusters_gray, pca_gray = realizar_pca_clusterizacao(
    imagens_gray, 
    n_componentes=50, 
    n_clusters=categorias_distintas//2)

# Adiciona os clusters ao DataFrame original
sample_items_meli_gray = sample_items_meli.copy()
sample_items_meli_gray['cluster'] = pd.Series(clusters_gray, index=sample_items_meli_gray.index)

# Exemplo: Obtém produtos similares para um item específico usando clusters em escala de cinza
produtos_similares_gray = obter_produtos_similares(sample_items_meli_gray, item_id_exemplo)
print(f"Produtos similares (Escala de Cinza) para o item {item_id_exemplo}:\n", produtos_similares_gray)

Produtos similares (Escala de Cinza) para o item MLB2897190943:
             item_id                                     ITE_ITEM_TITLE  \
1408  MLB1973969919  Babuche Adulto Infantil Sandalia  Ótima Qualid...   
2222  MLB3755618029  Chinelo Pegada Ortopédico Esporão Amortecedor ...   
2487  MLB4444627426  Colchão Inflável Casal Resistente 300kg + Bomb...   
3849  MLB3441141957  Tênis Olympikus Treino Dynamic Color Marinho/d...   
3890  MLB2189958055  Tenis Feminino Academia Ultra Leve Olympikus A...   

     ITE_ITEM_DOM_DOMAIN_ID  buyer_ID  \
1408  MLB-SANDALS_AND_CLOGS    132073   
2222  MLB-SANDALS_AND_CLOGS    201225   
2487     MLB-AIR_MATTRESSES    224129   
3849           MLB-SNEAKERS    318272   
3890           MLB-SNEAKERS    321378   

                                     thumbnail_path  cluster  
1408  item_thumbs/790053-MLB74329168177_012024.jpeg      378  
2222  item_thumbs/881567-MLB77028202326_062024.jpeg      378  
2487  item_thumbs/847773-MLU74931047398_032024.jpeg   

In [197]:
# === Recomendação usando SVD nas categorias compradas por cada usuário ===
# Cria uma matriz de compra de usuário por categoria
matriz_usuario_categoria = sample_items_meli.pivot_table(
    index='buyer_ID',
    columns='ITE_ITEM_DOM_DOMAIN_ID',
    aggfunc='size',
    fill_value=0
)

# Realiza a decomposição SVD
svd = TruncatedSVD(n_components=50, random_state=42)
fatores_usuario = svd.fit_transform(matriz_usuario_categoria)

# Função para recomendar categorias baseadas em usuários similares e exibir as categorias já compradas
def recomendar_categorias(usuario_id, fatores_usuario, matriz_usuario_categoria, top_n=5):
    # Obtém o índice do usuário na matriz
    idx_usuario = matriz_usuario_categoria.index.get_loc(usuario_id)
    # Obtém o vetor de fatores do usuário
    vetor_usuario = fatores_usuario[idx_usuario].reshape(1, -1)
    # Calcula a similaridade coseno entre o usuário e todos os outros usuários
    similaridades = cosine_similarity(vetor_usuario, fatores_usuario)[0]
    # Ordena os usuários pela similaridade (excluindo o próprio usuário)
    idx_usuarios_similares = np.argsort(-similaridades)[1:]
    usuarios_similares = matriz_usuario_categoria.index[idx_usuarios_similares]
    # Soma as categorias compradas pelos usuários similares
    categorias = matriz_usuario_categoria.loc[usuarios_similares].sum()
    # Remove as categorias que o usuário já comprou
    categorias_usuario = matriz_usuario_categoria.loc[usuario_id]
    categorias_recomendar = categorias[categorias_usuario == 0]
    # Recomenda as top_n categorias
    categorias_recomendadas = categorias_recomendar.sort_values(ascending=False).head(top_n).index.tolist()
    
    # Filtra as categorias já compradas pelo usuário
    categorias_ja_compradas = categorias_usuario[categorias_usuario > 0].index.tolist()
    
    return categorias_recomendadas, categorias_ja_compradas

# Exemplo: Recomenda categorias para um usuário específico e imprime as categorias já compradas
usuario_id_exemplo = matriz_usuario_categoria.index[0]
categorias_recomendadas, categorias_ja_compradas = recomendar_categorias(usuario_id_exemplo, fatores_usuario, matriz_usuario_categoria)

print(f"Categorias já compradas pelo usuário {usuario_id_exemplo}:\n", categorias_ja_compradas)
print(f"Categorias recomendadas para o usuário {usuario_id_exemplo}:\n", categorias_recomendadas)


Categorias já compradas pelo usuário 6:
 ['MLB-BICYCLE_ACCESSORIES_AND_REPLACEMENTS', 'MLB-PORTABLE_HANGING_SCALES', 'MLB-WATER_HOSES']
Categorias recomendadas para o usuário 6:
 ['MLB-SUPPLEMENTS', 'MLB-SNEAKERS', 'MLB-SANDALS_AND_CLOGS', 'MLB-PANTS', 'MLB-HAIR_TREATMENTS']
